In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

# Basic Text Generation 

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model='gpt-5-mini',
    input='Write a one sentence prediction for the future of intelligence.'
)

print(response.output_text)

In the coming decades, intelligence will evolve from isolated minds into networked human–machine collectives that dramatically amplify creativity and problem‑solving while forcing the creation of new ethical frameworks and governance to manage their power.


# Analyse Images and Files

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "role": "user",
            "content": [
             {
                 "type": "input_text",
                 "text": "What is in this image?"
             },
             {
                 "type": "input_image",
                 "image_url": "https://images.unsplash.com/photo-1464983953574-0892a716854b",
              }
             ]
         }
    ]
)


print(response.output_text)

A starry night landscape with a rugged, snow-capped mountain range. The foreground has patches of snow on dark slopes, and faint clouds sit low under a clear, starlit sky.


In [4]:
from IPython.display import display, HTML

html_code = """
<div style="display: flex; flex-direction: column; align-items: center; margin-top: 24px;">
  <img src="https://images.unsplash.com/photo-1464983953574-0892a716854b" alt="Mountain Lake" style="max-width: 600px; width: 100%; border-radius: 10px; box-shadow: 0 2px 16px rgba(0,0,0,0.18);">
  <p style="margin-top: 10px; color: #444; font-size: 1.1em;">A beautiful mountain lake. (Photo credit: Unsplash)</p>
</div>
"""

display(HTML(html_code))

# Structured Outputs

In [6]:
from pydantic import BaseModel

class Tasks(BaseModel):
    tasks: list[str]
    deadlines: list[str]

meeting_notes = """
Yesterday we discussed several action items:
• John needs to review the Q2 budget proposal before the end of this week.
• Let's set up a kickoff meeting for the new project—how about next Monday?
• Anna will update and submit the marketing plan by April 15th.
Please make sure to update the status before our next check-in.
"""
response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {
            "role": "system",
            "content": "You Extract tasks and their deadlines from raw inputs."
        },
        {
            "role": "user",
            "content": meeting_notes
         }
    ],
    text_format=Tasks,
)

print(response.output_parsed)

tasks=['John needs to review the Q2 budget proposal', 'Set up a kickoff meeting for the new project', 'Anna will update and submit the marketing plan', 'Update the status'] deadlines=['before the end of this week', 'next Monday', 'April 15th', 'before our next check-in']


In [7]:
from IPython.display import display, Markdown

# Nicely parse and render the tasks/deadlines as a markdown checklist for students
if hasattr(response, "output_parsed") and response.output_parsed:
    tasks = response.output_parsed.tasks
    deadlines = response.output_parsed.deadlines
    if tasks and deadlines and len(tasks) == len(deadlines):
        md_lines = ["### 📋 Extracted Tasks", ""]
        for task, deadline in zip(tasks, deadlines):
            md_lines.append(f"- [ ] **{task}**  \n  _Deadline: {deadline}_")
        display(Markdown("\n".join(md_lines)))
    else:
        display(Markdown("Could not match tasks and deadlines."))
else:
    display(Markdown("Could not parse tasks from the response."))

### 📋 Extracted Tasks

- [ ] **John needs to review the Q2 budget proposal**  
  _Deadline: before the end of this week_
- [ ] **Set up a kickoff meeting for the new project**  
  _Deadline: next Monday_
- [ ] **Anna will update and submit the marketing plan**  
  _Deadline: April 15th_
- [ ] **Update the status**  
  _Deadline: before our next check-in_

# Function Calling/Tool Calling

In [5]:
from openai import OpenAI
import json

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "load_pdf_text",
        "description": "Load and return the full text from the provided PDF file path.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "Path to the PDF file to extract text from.",
                },
            },
            "required": ["file_path"],
        },
    },
]

def load_pdf_text(file_path):
    """Loads text from a PDF file."""
    from pypdf import PdfReader
    reader = PdfReader(file_path)
    text = "\n\n".join([page.extract_text() for page in reader.pages])
    return text

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "Can you read and summarize the file at ../assets/paper3.pdf?"}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

response.output

[ResponseReasoningItem(id='rs_08eb90013507691800695d31d81b508197b3e82fa6bff68652', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionToolCall(arguments='{"file_path":"../assets/paper3.pdf"}', call_id='call_rkYEBNzgLDZQtoYdrtMnA2L3', name='load_pdf_text', type='function_call', id='fc_08eb90013507691800695d31da18108197adeb849a6939dcf9', status='completed')]

In [ ]:
for item in response.output:
    if item.type == "function_call":
        if item.name == "load_pdf_text":
            # 3. Execute the function logic for load_pdf_text
            file_args = json.loads(item.arguments)
            pdf_text = load_pdf_text(file_args["file_path"])

            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "pdf_text": pdf_text
                })5
            })2
print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
2    instructions="Summarize the PDF content provided via the tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

<img src="../assets/functions_schema.png" style="width: 50%;" />

https://platform.openai.com/docs/guides/function-calling#defining-functions

# Managing Conversation States

In [8]:
from openai import OpenAI
from openai.types.shared import reasoning_effort

client = OpenAI()

input_prompt = "What are the 3 core skills for snowboarding?"

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {"role": "user", "content": input_prompt},
    ],
    
)

print(response.output_text)

The 3 core skills for snowboarding are:

1. **Balance and Stance** – Maintaining proper body position and weight distribution on the board to stay stable and in control.
2. **Turning** – Learning how to carve turns using edge control to navigate the slopes smoothly.
3. **Stopping** – Being able to slow down and come to a controlled stop safely, often using techniques like the "heel-side stop" or "toe-side stop."

Mastering these skills helps build a strong foundation for progressing in snowboarding.


In [9]:
# Add the response to the conversation
history = [
        {"role": "user", "content": input_prompt},
    ]
history += [{"role": el.role, "content": el.content} for el in response.output]
history

[{'role': 'user', 'content': 'What are the 3 core skills for snowboarding?'},
 {'role': 'assistant',
  'content': [ResponseOutputText(annotations=[], text='The 3 core skills for snowboarding are:\n\n1. **Balance and Stance** – Maintaining proper body position and weight distribution on the board to stay stable and in control.\n2. **Turning** – Learning how to carve turns using edge control to navigate the slopes smoothly.\n3. **Stopping** – Being able to slow down and come to a controlled stop safely, often using techniques like the "heel-side stop" or "toe-side stop."\n\nMastering these skills helps build a strong foundation for progressing in snowboarding.', type='output_text', logprobs=[])]}]

In [10]:
second_input = {"role": "user", "content": "For each of these give me 3 exercises to practice in simple bullets."}

history.append(second_input)

second_response = client.responses.create(
    model="gpt-4.1-mini",
    input=history,
)

print(second_response.output_text)

Certainly! Here are 3 exercises for each core snowboarding skill:

**1. Balance and Stance**  
- Practice standing on a balance board or wobble board for 1-2 minutes to improve stability.  
- Skateboard or use a longboard to simulate balance and stance while moving on flat ground.  
- On the snow, practice “static stance” by standing still on your snowboard with knees slightly bent and eyes forward for 30 seconds at a time.

**2. Turning**  
- Practice “falling leaf” drills: slide down the slope sideways while shifting your weight to make gentle heel-side or toe-side turns without completely turning down the hill.  
- Use a gentle slope to practice linking heel-side to toe-side turns smoothly in a controlled, slow manner.  
- Practice “S turns” on a mild slope, focusing on using your edges to carve clean, rounded turns both directions.

**3. Stopping**  
- Practice the “falling leaf stop”: start sliding sideways and gradually dig in your heel or toe edge to come to a controlled stop.  

# Conversations API

To facilitate conversation management and avoid having to pass inputs manually like this, OpenAI created a special API to work with the Responses API called: "Conversations API".

The [Conversations API](https://platform.openai.com/docs/api-reference/conversations/create) works with the Responses API to persist conversation state as a long-running object with its own durable identifier. After creating a conversation object, you can keep using it across sessions, devices, or jobs.

Conversations store items, which can be messages, tool calls, tool outputs, and other data.

In [11]:

from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="What are the 3 most important skills to develop as a programmer.",
)
print(response.output_text)

second_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id,
    input=[{"role": "user", "content": "Write down 2 exercises for each of those skills."}],
)
print(second_response.output_text)

As a programmer, there are many skills that can enhance your effectiveness and career prospects. However, the three most important skills to develop are:

1. **Problem-Solving Skills**:
   - The core of programming is troubleshooting and solving complex problems. Being able to break down problems into smaller, manageable parts and devise solutions is crucial. Enhance this skill through practice, algorithms, and coding challenges.

2. **Coding Proficiency in Relevant Languages**:
   - Mastering one or more programming languages is essential. Understanding syntax, libraries, and frameworks related to your chosen languages allows you to write efficient and effective code. Focus on languages relevant to your field, such as Python, Java, or JavaScript, and keep learning new tools and technologies.

3. **Version Control and Collaboration Skills**:
   - Familiarity with version control systems like Git is critical in modern software development. Being able to collaborate effectively with othe

In [ ]:
from openai import OpenAI

client = OpenAI()

# iniitliaze a conversation session with an ID
def create_conversation(user_id: str) -> str:
    # 1. Create a new conversation container
    conversation = client.conversations.create(
        metadata={"user_id": user_id, "status": "active"}
    )
    return conversation.id

# something to send messages
def send_message(conv_id: str, user_text: str) -> str:
    # 2. Send a turn through Responses, bound to that conversation
    response = client.responses.create(
        model="gpt-5-mini",
        input=[{"role": "user", "content": user_text}],
        conversation=conv_id,
        # store=True  # optional; default is stored for conversation state
    )
    return response.output_text

# ability to update conversation metadata
def update_tag_conversation(conv_id: str, topic: str) -> None:
    # 3. Update metadata on the conversation (for filtering, dashboards, etc.)
    client.conversations.update(
        conversation_id=conv_id,
        metadata={"topic": topic}
    )

user_id = "user-123"
conv_id = create_conversation(user_id)

msg = "Hi, I'm Lucas and I want help deciding on my new laptop. \
    Give me just a sentence with a simple decision making framework."
answer = send_message(conv_id, msg)
print("Assistant:", answer)

# maybe after inspecting the first message:
update_tag_conversation(conv_id, topic="shopping-help")
msg = "What's my name and what am I buying?"
followup = send_message(conv_id, msg)
print("Assistant:", followup)

# Managing Context

![Context window illustration](https://cdn.openai.com/API/docs/images/context-window.png)


- Context is managed in tokens
- Tokens are just simple units of text
- [Context window limits for each OpenAI Model](https://platform.openai.com/docs/models)

In [12]:
import tiktoken

def get_num_tokens(prompt, model="gpt-5"):
    """Calculates the number of tokens in a text prompt"""
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(prompt))

In [13]:
get_num_tokens("Hello folks!")

3

## Token Cost & Context Windows for GPT-5-... models

__GPT-5.2__
- https://platform.openai.com/docs/models/gpt-5.2

__GPT-5.1__
- https://platform.openai.com/docs/models/gpt-5.1

__GPT-5__
- https://platform.openai.com/docs/models/gpt-5 

__GPT-5-mini__
- https://platform.openai.com/docs/models/gpt-5-mini